Project 2 Hypothese Testing

Topic: Amazon's Top 50 bestselling books from 2009 to 2019. Contains 550 books, data has been categorized into fiction and non-fiction using Goodreads. I would do some hypotheses test on ranking and evaluate books differently depending on the genre or evaluate if there is a trend toward darker themes in more recent years. This dataset named “bestsellers with categories.csv” is found in https://www.kaggle.com/sootersaalu/amazon-top-50-bestselling-books-2009-2019

In [163]:
#Import data
import os
print(os.getcwd())

/Users/lingzhang/Documents/Spring2021/DA 601


In [164]:
import pandas as pd
import numpy as np
import seaborn as sns
from numpy import nan
import missingno as msno
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import collections
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import plotly.figure_factory as ff
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu

In [165]:
#use pandas to read a csv file into DataFrame
df = pd.read_csv('/Users/lingzhang/Documents/Spring2021/DA 601/bestsellers with categories.csv')
df.head(10)

,Name,Author,User Rating,Reviews,Price,Year,Genre
0,10-Day Green Smoothie Cleanse,JJ Smith,4.7,17350,8,2016,Non Fiction
1,11/22/63: A Novel,Stephen King,4.6,2052,22,2011,Fiction
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,4.7,18979,15,2018,Non Fiction
3,1984 (Signet Classics),George Orwell,4.7,21424,6,2017,Fiction
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,4.8,7665,12,2019,Non Fiction
5,A Dance with Dragons (A Song of Ice and Fire),George R. R. Martin,4.4,12643,11,2011,Fiction
6,A Game of Thrones / A Clash of Kings / A Storm...,George R. R. Martin,4.7,19735,30,2014,Fiction
7,A Gentleman in Moscow: A Novel,Amor Towles,4.7,19699,15,2017,Fiction
8,"A Higher Loyalty: Truth, Lies, and Leadership",James Comey,4.7,5983,3,2018,Non Fiction
9,A Man Called Ove: A Novel,Fredrik Backman,4.6,23848,8,2016,Fiction


In [166]:
df.info

<bound method DataFrame.info of                                                   Name  \
0                        10-Day Green Smoothie Cleanse   
1                                    11/22/63: A Novel   
2              12 Rules for Life: An Antidote to Chaos   
3                               1984 (Signet Classics)   
4    5,000 Awesome Facts (About Everything!) (Natio...   
..                                                 ...   
545       Wrecking Ball (Diary of a Wimpy Kid Book 14)   
546  You Are a Badass: How to Stop Doubting Your Gr...   
547  You Are a Badass: How to Stop Doubting Your Gr...   
548  You Are a Badass: How to Stop Doubting Your Gr...   
549  You Are a Badass: How to Stop Doubting Your Gr...   

                       Author  User Rating  Reviews  Price  Year        Genre  
0                    JJ Smith          4.7    17350      8  2016  Non Fiction  
1                Stephen King          4.6     2052     22  2011      Fiction  
2          Jordan B. Peterson  

In [167]:
#deduce data types
df.dtypes

Name            object
Author          object
User Rating    float64
Reviews          int64
Price            int64
Year             int64
Genre           object
dtype: object

In [168]:
# Displaying measures of the central trend
df.describe().T

,count,mean,std,min,25%,50%,75%,max
User Rating,550.0,4.618364,0.226980,3.3,4.5,4.7,4.80,4.9
Reviews,550.0,11953.281818,11731.132017,37.0,4058.0,8580.0,17253.25,87841.0
Price,550.0,13.100000,10.842262,0.0,7.0,11.0,16.00,105.0
Year,550.0,2014.000000,3.165156,2009.0,2011.0,2014.0,2017.00,2019.0


In [169]:
# Take all authors with the highest average rating
top_10_autors = bestsellers.groupby('Author')[['User Rating']].mean()\
                                                              .sort_values('User Rating', ascending=False)\
                                                              .head(10)\
                                                              .reset_index()
# Take the authors who have written the most bestsellers
number_of_books_written = bestsellers.groupby('Author')[['Name']].count()\
                                                                 .sort_values('Name', ascending=False)\
                                                                 .head(10)\
                                                                 .reset_index()

# Take books with the number of reviews
books_with_the_number_of_reviews = bestsellers.groupby('Name')[['Reviews']].sum()\
                                                                           .sort_values('Reviews', ascending=False)\
                                                                           .head(5)\
                                                                           .reset_index()

# Take the number of books by genre
number_of_books_by_genre = bestsellers.groupby('Genre')[['Name']].count()\
                                                                 .sort_values('Name', ascending=False)\
                                                                 .head(10)\
                                                                 .reset_index()

In [170]:
# Set the visualization parameters
# Set the number of rows and columns
#Reference URL https://plotly.com/python/reference/layout/
fig_1 = make_subplots(rows=1, cols=1, subplot_titles=("")) 


# Setting table parameters
fig_1.add_trace(go.Table(columnorder = [1,2],
                         columnwidth = [100,50],
                         header=dict(values=top_10_autors.columns, 
                                     line_color='black',
                                     fill_color='LightSkyBlue',
                                     height=30),
                         cells=dict(values=[top_10_autors['Author'],
                                    top_10_autors['User Rating']], 
                                    line_color='darkslategray',
                                    fill_color='White')))

# Setting the parameters of the chart when displaying
fig_1.update_layout(showlegend=False, 
                    title_text='Top 10 best authors', 
                    title_font_size=16, 
                    title_font_family='Arial', 
                    title_x=0.5,
                    font=dict(family='Arial', 
                              size=13, 
                              color='black'))
fig_1.show()

In [171]:
#Set the visualization parameters
#References url: https://plotly.com/python/marker-style/
fig_2 = make_subplots(rows=1, cols=2,
                      specs=[[{'type': 'xy'}, {"type": "xy"}]],
                      subplot_titles=("Top 10 authors by number of bestsellers", 
                                      "Top 5 books by number of reviews"))

# Setting Bar parameters
fig_2.add_trace(go.Bar(x=number_of_books_written['Author'], 
                       y=number_of_books_written['Name'],
                       name ='Count books',
                       marker_color='LightSkyBlue'),
                       row=1, col=1)

# Setting Bar parameters
fig_2.add_trace(go.Bar(x=books_with_the_number_of_reviews['Name'], 
                       y=books_with_the_number_of_reviews['Reviews'],
                       marker_color='#C8B9EE',
                       name ='Reviews'),
                       row=1, col=2)

# Setting the parameters of the chart when displaying
fig_2.update_traces(marker_line_color='Gray',
                    marker_line_width=1)

# Setting the parameters of the chart when displaying
fig_2.update_layout(showlegend=False, 
                    template='simple_white', 
                    font=dict(family='Arial', 
                              size=12, 
                              color='black'))
fig_2.show()


In [172]:
 # Let's set the visualization parameters
fig_3 = make_subplots(rows=1, cols=1,
                      specs=[[{'type': 'domain'}]],
                      subplot_titles=(""))

# Setting Pie parameters
fig_3.add_trace(go.Pie(values=number_of_books_by_genre['Name'],
                       title='Genre',
                       labels=['Non Fiction', 'Fiction'],
                       textinfo='label+percent',
                       insidetextorientation='radial',
                       hole=0.3,
                       marker=dict(colors=['MidnightBlue','LightSkyBlue'], 
                                   line_color='Gray', 
                                   line_width=1),
                       textfont={'color': '#FFFFFF', 'size': 15},
                       textfont_size=14))

# Setting the parameters of the chart when displaying
fig_3.update_layout(margin_t=30, 
                    showlegend=False, 
                    title_text='Count of books genre', 
                    title_x=0.5, 
                    font=dict(family='Arial', 
                              size=14, 
                              color='black'))
fig_3.show() 

In [173]:
fig_4 = make_subplots(rows=1, cols=1, specs=[[{'type': 'xy'}]])

# Setting Box parameters
fig_4.add_trace(go.Box(x=bestsellers['User Rating'], 
                       name='User Rating'))

# Setting the parameters of the Box when displaying
fig_4.update_traces(marker_color='LightSkyBlue')

# Setting the parameters of the Box when displaying
fig_4.update_layout(showlegend=False, 
                    template='simple_white', 
                    font=dict(family='Arial', 
                              size=12, 
                              color='black'))
fig_4.show()

Using formuler to do the hypotheses test

H0 - No differences between genre racketeering
<br>H1 - Does have differences between genre racketeering

In [174]:
# Set the alpha level
alpha=0.05

# We pass groups to the criterion for testing
stat, pval = shapiro(bestsellers['User Rating'])

print('Statistic:', f'{stat:.3f}')
print('P-Value:', f'{pval:.20f}')
    
# Checking the condition for accepting or rejecting H0
if pval > alpha:
    print('Accept H0 - Data is distributed normally.')
if pval < alpha:
    print('Reject H0 - Data is not distributed normally.')

Statistic: 0.877
P-Value: 0.00000000000000040170
Reject H0 - Data is not distributed normally.


In [175]:
# Trying two samples for testing
# Create group

non_fiction = df[df['Genre'] == 'Non Fiction']['User Rating']
fiction = df[df['Genre'] == 'Fiction']['User Rating']

In [176]:
#Since the data in the samples are not normally distributed
#Trying nonparametric Mann-Whitney test
#Reference URL: https://machinelearningmastery.com/nonparametric-statistical-significance-tests-in-python/
#https://towardsdatascience.com/determine-if-two-distributions-are-significantly-different-using-the-mann-whitney-u-test-1f79aa249ffb

# Set the alpha level
alpha=0.05

# We pass groups to the criterion for testing
stat, pval = mannwhitneyu(non_fiction, fiction)

print('Statistic:', f'{stat:.3f}')
print('P-Value:', f'{pval:.20f}')
 
# Checking the condition for accepting or rejecting H0
if pval > alpha:
    print('Accept H0 - There is no significant difference between the groups.')
if pval < alpha:
    print('Reject H0 - There are significant differences between the groups.')


Statistic: 27490.500
P-Value: 0.00000004857354068641
Reject H0 - There are significant differences between the groups.


In [177]:
#compare the median values in the groups.
print(f'Non_fiction median: {non_fiction.median()}, Fiction median: {fiction.median()}')

Non_fiction median: 4.6, Fiction median: 4.7


In [178]:
#Visualizing the density of distribution in samples
#Create distplot with curve_type set to 'normal'
fig_5 = ff.create_distplot([non_fiction, fiction], 
                         ['Non_fiction', 'Fiction'], 
                         colors=['LightSkyBlue', 'gray',],
                         bin_size=.1, 
                         show_rug=False)

# Add title
fig_5.update_layout(title_text='Сomparison of the distribution density in the group Non_fiction and the group', 
                     template='simple_white',
                     font=dict(family='Arial', size=12, color='black'))
fig_5.show()

As a result, we can see that users evaluate books differently depending on the genre, and based on the median values, readers give preference to works of fiction

References URL:<br>
<br> https://towardsdatascience.com/string-matching-with-fuzzywuzzy-e982c61f8a84
<br> https://www.programcreek.com/python/example/100283/plotly.tools.make_subplots
<br> https://codingnomads.co/blog/data-analysis-example-analyzing-movie-ratings-with-python/
<br>https://www.programcreek.com/python/example/121873/plotly.subplots.make_subplots
<br>https://machinelearningmastery.com/nonparametric-statistical-significance-tests-in-python/
<br>https://towardsdatascience.com/determine-if-two-distributions-are-significantly-different-using-the-mann-whitney-u-test-1f79aa249ffb
<br> https://codingnomads.co/blog/data-analysis-example-analyzing-movie-ratings-with-python/
https://www.geeksforgeeks.org/fuzzywuzzy-python-library/
<br>https://anaconda.org/conda-forge/plotly
<br> https://plotly.com/python/reference/layout/